# Predicting the price of a house in budapest using Machine learning 

### 1.1 getting the data
- All of the data here was collected with the `ingatlan_com_webscraping.py` script from https://ingatlan.com
- The program is made for scraping houses within specific districts
- The collection of data takes a significant time, due to bot detection (approx. 6 hours for each district)

In [2]:
import pandas as pd
import numpy as np 


In [3]:
first_second_third_part_1 = pd.read_csv("datasets/elso_masodik_harmadik_ker.csv")
first_second_third_part_2 = pd.read_csv("datasets/elso_masodik_harmadik_ker_ketto.csv")
first_second_third = pd.concat([first_second_third_part_1,first_second_third_part_2])
fourth_fifth_part_1 = pd.read_csv("datasets/negy_ot_egy.csv")
fourth_fifth_2 = pd.read_csv("datasets/negy_ot_ketto.csv")
fourth_fifth = pd.concat([fourth_fifth_part_1,fourth_fifth_2])
sixth = pd.read_csv("datasets/hatodik_ker.csv")
seventh = pd.read_csv("datasets/7ker_osszes.csv")
eigth_pt_1 = pd.read_csv("datasets/nyolcadik_ker.csv")
eigth_pt_2 = pd.read_csv("datasets/nyolcadik_ker_ketto.csv")
eigth = pd.concat([eigth_pt_1,eigth_pt_2])
ninth = pd.read_csv("datasets/kilencedik_ker.csv")
tenth = pd.read_csv("datasets/tizedik_ker.csv")
eleventh = pd.read_csv("datasets/tizenegyedik_ker_elso.csv")


In [43]:
#put all of the data together 
budapest_houses = pd.concat([first_second_third,fourth_fifth, sixth, seventh,eigth,ninth,tenth,eleventh])
budapest_houses.head()

,price,adress,size,rooms,half_rooms,balcony,Ingatlan állapota,Építés éve,Komfort,Emelet,...,Erkély mérete,Kertkapcsolatos,Tetőtér,Parkolás,Parkolóhely ára,Fűtés,Szigetelés,Energiatanúsítvány,Panelprogram,Energetikai tanúsítvány
0,57.3,"III. kerület, Pünkösdfürdő utca",44.0,2.0,0.0,4.18,új építésű,2023,duplakomfortos,7,...,4.18 m|2|,nem,nem tetőtéri,teremgarázs hely - megvásárolható,"5,50 M Ft",házközponti egyedi méréssel,|nincs megadva|,|nincs megadva|,NaN,NaN
1,765.0,"II. kerület, Bem rakpart",150.0,4.0,0.0,NaN,felújított,1950 előtt,luxus,2,...,|nincs megadva|,|nincs megadva|,nem tetőtéri,|nincs megadva|,NaN,"gáz (konvektor), egyéb",|nincs megadva|,|nincs megadva|,NaN,NaN
2,199.0,"II. kerület, Pasaréti út 84b",98.0,3.0,0.0,34.00,újszerű,2012,összkomfortos,3,...,34 m|2|,igen,nem tetőtéri,teremgarázs hely - megvásárolható,20 M Ft,házközponti egyedi méréssel,van - 20 cm,BB,NaN,NaN
3,40.9,"III. kerület, Juhász Gyula utca 2.",51.0,2.0,0.0,4.00,felújított,1950 és 1980 között,összkomfortos,4,...,4 m|2|,nem,nem tetőtéri,"utca, közterület - ingyenes",NaN,"távfűtés egyedi méréssel, hűtő-fűtő klíma",nincs,|nincs megadva|,nem vett részt,NaN
4,64.9,"III. kerület, Váradi utca",66.0,3.0,0.0,6.00,felújított,|nincs megadva|,összkomfortos,3,...,6 m|2|,nem,nem tetőtéri,"utca, közterület - fizetős övezet",NaN,távfűtés egyedi méréssel,nincs,|nincs megadva|,nem vett részt,NaN


## 1.2 filtering the relevant features 

In [44]:
print(budapest_houses.columns)

Index(['price', 'adress', 'size', 'rooms', 'half_rooms', 'balcony',
       'Ingatlan állapota', 'Építés éve', 'Komfort', 'Emelet',
       'Épület szintjei', 'Lift', 'Belmagasság', 'Légkondicionáló',
       'Akadálymentesített',
       'Átlag gázfogyasztás\n                            |info|',
       'Átlag áramfogyasztás\n                            |info|',
       'Rezsiköltség', 'Közös költség', 'Fürdő és wc', 'Tájolás', 'Kilátás',
       'Erkély mérete', 'Kertkapcsolatos', 'Tetőtér', 'Parkolás',
       'Parkolóhely ára', 'Fűtés', 'Szigetelés', 'Energiatanúsítvány',
       'Panelprogram', 'Energetikai tanúsítvány'],
      dtype='object')


In [67]:
selected_columns = [
    'price', 'adress', 'size', 'rooms', 'half_rooms', 'balcony',
    'Ingatlan állapota', 'Építés éve', 'Komfort', 'Emelet',
     'Lift', 'Légkondicionáló',
    'Fürdő és wc', 'Tájolás', 'Kilátás',
    
]
budapest_houses_selected = budapest_houses[selected_columns]
budapest_houses_selected["Légkondicionáló"]

0       |nincs megadva|
1                   van
2                   van
3                   van
4                 nincs
             ...       
1835              nincs
1836                van
1837    |nincs megadva|
1838                van
1839    |nincs megadva|
Name: Légkondicionáló, Length: 16960, dtype: object

In [68]:
column_translation = {
    'price': 'price',
    'adress': 'address',
    'size': 'size',
    'rooms': 'rooms',
    'half_rooms': 'half_rooms',
    'balcony': 'balcony',
    'Ingatlan állapota': 'property_condition',
    'Építés éve': 'construction_year',
    'Komfort': 'comfort',
    'Emelet': 'floor',
    'Lift': 'elevator',
    'Légkondicionáló': 'air_conditioning',
    'Fürdő és wc': 'bathroom_and_wc',
    'Tájolás': 'orientation',
    'Kilátás': 'view',
    
}

# Rename columns using the translation dictionary
budapest_houses_selected = budapest_houses_selected.rename(columns=column_translation)
budapest_houses_selected

,price,address,size,rooms,half_rooms,balcony,property_condition,construction_year,comfort,floor,elevator,air_conditioning,bathroom_and_wc,orientation,view
0,57.3,"III. kerület, Pünkösdfürdő utca",44.0,2.0,0.0,4.18,új építésű,2023,duplakomfortos,7,van,|nincs megadva|,külön helyiségben,észak,utcai
1,765.0,"II. kerület, Bem rakpart",150.0,4.0,0.0,NaN,felújított,1950 előtt,luxus,2,van,van,külön helyiségben,kelet,panorámás
2,199.0,"II. kerület, Pasaréti út 84b",98.0,3.0,0.0,34.00,újszerű,2012,összkomfortos,3,van,van,külön és egyben is,délkelet,panorámás
3,40.9,"III. kerület, Juhász Gyula utca 2.",51.0,2.0,0.0,4.00,felújított,1950 és 1980 között,összkomfortos,4,van,van,külön helyiségben,nyugat,panorámás
4,64.9,"III. kerület, Váradi utca",66.0,3.0,0.0,6.00,felújított,|nincs megadva|,összkomfortos,3,nincs,nincs,külön helyiségben,|nincs megadva|,kertre néz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1835,58.0,"XI. kerület, Tas vezér utca",43.0,1.0,0.0,2.00,felújított,1950 és 1980 között,összkomfortos,földszint,van,nincs,egy helyiségben,délkelet,kertre néz
1836,105.0,"XI. kerület, Hadak útja 5.",108.0,4.0,1.0,20.00,újszerű,2022,duplakomfortos,2,van,van,külön helyiségben,délkelet,udvari
1837,92.9,"XI. kerület, Napkelte utca",49.0,2.0,0.0,7.53,újszerű,2019,|nincs megadva|,1,van,|nincs megadva|,külön helyiségben,dél,utcai
1838,63.9,"XI. kerület, Alíz utca",37.0,1.0,1.0,4.00,jó állapotú,2019,|nincs megadva|,7,van,van,|nincs megadva|,délnyugat,utcai


## 1.3 getting the data ready 

In [69]:
HAS_ENCODING_FOR_AC = {
    "van" : True,
    "nincs" : False,
    "|nincs megadva|" : False
    }
HAS_ENCODING_FOR_ELEVATOR = {
    "van" : True,
    "nincs" : False
    }    


def Floor_Parse(x):
    if "földszint" in str(x):
        return 0
    else:
        try:
            return float(x)
        except:
            pass
def Construction_Year_Parse(x):
    x = str(x).split()
    if len(x) > 2:
        try:
            return (float(x[0]) + float(x[2])) / 2
        except:
            pass
    elif len(x) > 1: 
        try:
            return float(x[0])
        except:
            pass
    elif len(x) == 1:
        try:
            return float(x[0])
        except:
            pass
def adress_parse_to_district(x):
    x = str(x).split()
    try:
        return x[0]
    except:
        pass


In [70]:
budapest_houses_selected["air_conditioning"] = budapest_houses_selected["air_conditioning"].map(HAS_ENCODING_FOR_AC)
budapest_houses_selected["elevator"] = budapest_houses_selected["elevator"].map(HAS_ENCODING_FOR_ELEVATOR)
#budapest_houses_selected["construction_year"] = budapest_houses_selected["construction_year"].apply(lambda x : Construction_Year_Parse(x))
budapest_houses_selected["balcony"].fillna(0,inplace=True)
budapest_houses_selected["address"] = budapest_houses_selected["address"].apply(lambda x : adress_parse_to_district(x))
budapest_houses_selected["floor"] = budapest_houses_selected["floor"].apply(lambda x : Floor_Parse(x))
budapest_houses_selected["floor"].fillna(0,inplace=True)
budapest_houses_selected["elevator"].fillna(False,inplace=True)

In [72]:

budapest_houses_selected.isna().sum()
budapest_houses_selected.dropna(inplace=True)


## 2.0 Finding the ideal machine learning model and training it 

In [75]:
X = budapest_houses_selected.drop("price", axis=1)
y = budapest_houses_selected["price"]

In [76]:
# 1. Import OneHotEncoder and ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
budapest_houses_selected.dropna()
# 2. Define the categorical features to transform
categorical_features = ["address", "property_condition","construction_year","comfort","elevator","air_conditioning","bathroom_and_wc","orientation","view"]

# 3. Create an instance of OneHotEncoder
one_hot = OneHotEncoder()

# 4. Create an instance of ColumnTransformer
transformer = ColumnTransformer([("one_hot", # name
                                  one_hot, # transformer
                                  categorical_features)], # columns to transform
                                  remainder="passthrough") # what to do with the rest of the columns? ("passthrough" = leave unchanged) 

# 5. Turn the categorical features into numbers (this will return an array-like sparse matrix, not a DataFrame)
transformed_X = transformer.fit_transform(X)
transformed_X

<16896x72 sparse matrix of type '<class 'numpy.float64'>'
	with 208548 stored elements in Compressed Sparse Row format>

In [79]:
len(X)

16896

### 2.1 Experimenting with Ridge model

In [80]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
X_train, X_test, y_train, y_test = train_test_split(transformed_X, y, test_size=0.2)
model = Ridge()
model.fit(X_train, y_train)

model.score(X_test, y_test)

0.7574691514591696

### 2.2 RandomForestRegressor

In [84]:
from sklearn.ensemble import RandomForestRegressor
np.random.seed(0)
X_train, X_test, y_train, y_test = train_test_split(transformed_X, y, test_size=0.2)
model = RandomForestRegressor()
model.fit(X_train, y_train)

model.score(X_test, y_test)

0.8182639676187492

### 2.3 saving the model 

In [85]:
import pickle

pickle.dump(model, open("random_forest_81_percent.pkl", "wb"))